In [1]:
import unicodedata, string, requests, pandas as pd
from time import time, sleep
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
def get_info(url, search_params, is_individual):
    agent_company = []
    
    for param in search_params:
        driver = webdriver.Chrome('./chromedriver.exe')
        driver.get(url)

        if is_individual:
            driver.find_element_by_xpath('//*[@id="rbl_SearchType_0"]').click()
            driver.find_element_by_xpath('//*[@id="ddl_SearchCriteria"]').send_keys('Agent Name')
        else:
            driver.find_element_by_xpath('//*[@id="rbl_SearchType_1"]').click()
            driver.find_element_by_xpath('//*[@id="ddl_SearchCriteria"]').send_keys('Company Name')
            
        driver.find_element_by_xpath('//*[@id="txt_SearchValue"]').send_keys(param)
        driver.find_element_by_xpath('//*[@id="btnSearch"]').click()
        sleep(1)

        result_html = driver.page_source
        driver.quit()

        results = BeautifulSoup(markup=result_html, features='html.parser')
        for table_data in results.find_all('tr', {'class':['griditem', 'gridalternatingitem']}):
            data = {
                'GIA No.': table_data('td')[0].text.strip(),
                'Name': table_data('td')[1].text.strip()
            }
            agent_company.append(data)
    
    agent_company_df = pd.DataFrame(agent_company)
    agent_company_df.drop_duplicates(inplace=True)
    
    return agent_company_df

def get_tsa(url, search_params):
    tsa = []

    for param in search_params:
        driver = webdriver.Chrome('./chromedriver.exe')
        driver.get(url)
        driver.find_element_by_xpath('//*[@id="ddlTSA"]').send_keys(param)
        sleep(1)
        result_html = driver.page_source
        driver.quit()

        results = BeautifulSoup(result_html, 'html.parser')

        for info in results.find_all('table', {'id':['dgAgentsList']}):
            for table_data in info.find_all('tr', {'class':['griditem','gridalternatingitem']}):
                data = {
                    'GIA No.':table_data('td')[0].text.strip(),
                    'Business Reg. No.':table_data('td')[1].text.strip(),
                    'Company Name':table_data('td')[2].text.strip(),
                    'Trade Type':param
                    }
                tsa.append(data)

    tsa_df = pd.DataFrame(tsa)
    tsa_df.drop_duplicates(inplace=True) 
    
    return tsa_df

def get_profile(url, gia_no):
    a_info = []
    n_info = []
    
    url = url+str(gia_no)
    req = requests.Session()
    sess = req.get(url)    
    content = BeautifulSoup(sess.content, 'html.parser')

    id_tag = ['AgentInfor_lblGIANoDetails', 'AgentInfor_lblAgentNameDetails', 'AgentInfor_lblAgentTypeDetails', \
                  'AgentInfor_lblTradeSpecificDetails', 'PrincipalInfor_lblPrincipalDetails']
        
    for data in content.find_all('table', {'id':['table1', 'table2']}):
        for info in data.find_all('span', {'id':id_tag}):
            a_info.append(unicodedata.normalize('NFKD', info.text.strip()))

    for data in content.find_all('tr', {'id':['trNomineeRecords']}):
        for info in data.find_all('tr', {'class':['griditem', 'gridalternatingitem']}):
            n_info.append(unicodedata.normalize('NFKD', info.text.strip()))

    return a_info, n_info

def check_text(text):
    text = text.replace('dP', 'd, P')  
    text = text.replace('d.P', 'd, P')
    text = text.replace('DP', 'D, P')
    text = text.replace('D.P', 'D, P')
    text = text.replace('hS', 'h. S')
    text = text.replace('tS', 't. S')    
    
    return text.replace('Non-', ', Non-')

# Individual Agent Information

In [3]:
search_params = list(string.ascii_lowercase)
url = 'https://www.arcm.com.sg/App/UI/Common/SearchAgentInfor.aspx?strApplicationType=SearchNominee'

start = time()
agents_df = get_info(url=url, search_params=search_params, is_individual=True)
print('Time taken: {}'.format(time() - start))
print(len(agents_df))

Time taken: 245.8126244544983
10782


In [4]:
agents_df.head(5)

GIA No.                         Name
0  C005180             ALICIA ENG SIMIN
1  A004917  ANG HUI PING (HONG HUIPING)
2  C005151                 Ang Wee Long
3  C001053           BRYAN ANDRE PAKIAM
4  A028403           CANDY GOH SIU PING

In [5]:
agents_df.to_excel('./individual.xlsx', index=False)
agents_df = pd.read_excel('./individual.xlsx')

In [6]:
url = 'https://www.arcm.com.sg/App/UI/Common/SearchAgentInfoDetails.aspx?GIANo='
agent_profile = []

start = time()
for e in range(len(agents_df)):
    d, n = get_profile(url=url, gia_no=agents_df['GIA No.'][e])
    data = {
        'Agent No':d[0],
        'Agency Name':d[1],
        'Type Of Trade':d[2],
        "Agent's principal":check_text(d[3]),
        'Nominee(s) Information':[check_text(text) for text in n]
    }
    agent_profile.append(data)
    if e % 1000 == 0:
        print('{} out of {} done.'.format(e+1, len(agents_df)))
print('Time taken: {}'.format(time() - start))

1 out of 10782 done.
1001 out of 10782 done.
2001 out of 10782 done.
3001 out of 10782 done.
4001 out of 10782 done.
5001 out of 10782 done.
6001 out of 10782 done.
7001 out of 10782 done.
8001 out of 10782 done.
9001 out of 10782 done.
10001 out of 10782 done.
Time taken: 2483.567063331604


In [7]:
agent_profile_df = pd.DataFrame(agent_profile)
len(agent_profile_df)

10782

In [8]:
agent_profile_df.to_excel('individual_details.xlsx', index=False)
individual = agents_df.join(other=agent_profile_df, how='left')
individual.to_excel('./individual_information.xlsx', index=False)

# Corporate Information

In [9]:
url = 'https://www.arcm.com.sg/App/UI/Common/SearchAgentInfor.aspx?strApplicationType=SearchNominee'

start = time()
corporate_df = get_info(url=url, search_params=search_params, is_individual=False)
print('Time taken: {}'.format(time() - start))
print(len(corporate_df))

Time taken: 216.8821256160736
2500


In [10]:
corporate_df.head(5)

GIA No.                       Name
0  C005544            1 Accurate Auto
1  C004619          1 ALTITUDE AGENCY
2  C004596   1 ASIA MANPOWER SERVICES
3  C004508            1 ASSIST AGENCY
4  C005002  1 INTEGRATED CARE PTE LTD

In [11]:
corporate_df.to_excel('./corporate.xlsx', index=False)
corporate_df = pd.read_excel('./corporate.xlsx')

In [12]:
url = 'https://www.arcm.com.sg/App/UI/Common/SearchAgentInfoDetails.aspx?GIANo='
corporate_profile = []

start = time()
for e in range(len(corporate_df)):
    d, n = get_profile(url=url, gia_no=corporate_df['GIA No.'][e])
    data = {
        'Agent No':d[0],
        'Company Name':d[1],
        'Type Of Trade':d[2],
        "Agent's principal":check_text(d[3]),
        'Nominee(s) Information':[check_text(text) for text in n]
    }
    corporate_profile.append(data)
    if e % 1000 == 0:
        print('{} out of {} done.'.format(e+1, len(corporate_df)))
print('Time taken: {}'.format(time() - start))

1 out of 2500 done.
1001 out of 2500 done.
2001 out of 2500 done.
Time taken: 355.05066752433777


In [13]:
corporate_profile_df = pd.DataFrame(corporate_profile)
len(corporate_profile_df)

2500

In [14]:
corporate_profile_df.to_excel('corporate_details.xlsx', index=False)
corporate = corporate_df.join(other=corporate_profile_df, how='left')
corporate.to_excel('./corporate_information.xlsx', index=False)

# Get TSA 

In [15]:
tsa_params = ['Card Protection Insurance', 'Electrical Protection Insurance', 'Freight Forwarders', \
              'Foreign Worker Agencies', 'Handphone Dealers', 'Maid Agencies', 'Motor Dealers', \
              'Maid Agencies + Foreign Worker Agencies', 'Travel Agents']

url = 'https://www.arcm.com.sg/App/UI/Common/TSA.aspx'

start = time()
tsa_df = get_tsa(url=url, search_params=tsa_params)
print('Time taken: {}'.format(time() - start))

Time taken: 62.986576080322266


In [16]:
tsa_df.head(5)

Business Reg. No.                                   Company Name  GIA No.  \
0        198203595R                           KAMIGUMI (S) PTE LTD  C001197   
1        197702167W  Panalpina World Transport (Singapore) Pte Ltd  C005218   
2        199506592H                Schenker (Asia Pacific) Pte Ltd  C005119   
3        197000914Z                   Schenker Singapore (Pte) Ltd  C004625   
4         53163757C                         AAKA NETWORK RESOURCES  C005213   

                Trade Type  
0       Freight Forwarders  
1       Freight Forwarders  
2       Freight Forwarders  
3       Freight Forwarders  
4  Foreign Worker Agencies

In [17]:
tsa_df.to_excel('./tsa.xlsx', index=False)
tsa_df = pd.read_excel('./tsa.xlsx')

In [18]:
print(len(tsa_df))

1412


In [19]:
url = 'https://www.arcm.com.sg/App/UI/Common/SearchAgentInfoDetails.aspx?GIANo='
profile = []

start = time()
for e in range(len(tsa_df)):
    d, n = get_profile(url=url, gia_no=tsa_df['GIA No.'][e])
    data = {
        'Agent No':d[0],
        'Company':d[1],
        'Type Of Trade':d[2],
        "Agent's principal":check_text(d[3]),
        'Nominee(s) Information':[check_text(text) for text in n]
    }
    profile.append(data)
    if e % 1000 == 0:
        print('{} out of {} done.'.format(e+1, len(tsa_df)))
print('Time taken: {}'.format(time() - start))

1 out of 1412 done.
1001 out of 1412 done.
Time taken: 250.48258805274963


In [20]:
tsa_profile_df = pd.DataFrame(profile)
len(tsa_profile_df)

1412

In [21]:
tsa_profile_df.to_excel('tsa_details.xlsx', index=False)
tsa = tsa_df.join(other=tsa_profile_df, how='left')
tsa.to_excel('./tsa_information.xlsx', index=False)